# Homeworks - Big Data and Public Policy Class

In [14]:
from IPython.display import display
import pandas as pd
pd.options.display.max_columns = None # Display all columns of a dataframe
pd.options.display.max_rows = 700
from pprint import pprint
import re

The objective of the following homework is to **bring to practice the tools studied in class** by conducting step by step a first big data project. We advise that you do it in parallel with the lectures: the relation between the project steps and the lectures is indicated below. 

You will build several **machine learning models that will predict continuous and categorical variables based on data coming from webscraping and/or an API**. 

We propose that you predict the **daily stock price data of a large company** (of your choice) **based on the company's tweets**. You can chose a company from one of the [major stock indices](https://www.wikiwand.com/en/List_of_stock_market_indices). 

---------------------------
After having decided the setting (your own or the company + corresponding timespan) that you will study, you should follow the following steps:

**Step 1a: `X` variables** [week 2]
- Fetch the data using the **twitter API** or any other API or website that you are interested in.
- Beware of the rate limits and organize your program so as to overcome them if needed
- The data should include some text, but might also have other interesting variables (retweets, favorites...)
- Create some (non-text based) `X_num` variables that you will use for the prediction

In [15]:
#this you get when you make create an application on twitter as a dev
consumer_key="7X8q1LteL1qOLRg4DSoiI0lyk"
consumer_secret="Bb08vV5XoxDEP4SGLJfkvEpuwvxEDvzVSRGTptzQCjC9XVmquP"
access_token="1230145588659998721-EQb8DoMYnorBtqAuEPMZtZcXumzuAe"
access_token_secret="mzurpOu1LWhj0NU9pctgYZ6OBzhQvyqw8hvRNpe3yI4Qc"

## Accessing the Twitter API using `tweepy`

We use the `tweepy` package (documentation: https://tweepy.readthedocs.io/en/latest/). Tweepy is an *An easy-to-use Python library for accessing the Twitter API.*

R users can use [rtweet](https://rtweet.info/), a similar package. 

Twitter requires all requests to use `OAuth` for authentication

In [16]:
import tweepy
from tweepy import OAuthHandler

Authenticate to Twitter

In [17]:
auth = OAuthHandler(consumer_key, consumer_secret) #creating an OAuthHandler instance
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

We specify `wait_on_rate_limit_notify==True` & `wait_on_rate_limit=True`. The API method will wait once you’ve reached your rate limit and prints out a message

In [23]:
# test authentication
try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


For an extensive list of the methods available, see the [API Reference page](https://tweepy.readthedocs.io/en/latest/api.html#api-reference). There are several types of methods. The following methods enable you to access twitter content:
- Timeline methods return a list of  `status` objects
- Status methods return a `status` object
- User methods return `user` object or a list of `user` objects. 
- Favorite methods: return a list of  `status` objects

For some methods, you can interact with twitter:
- Friendship Methods return a `user` object, by example:
    - `create_friendship`: creates a new friendship with the specified user ()
    
Let's review some useful methods:

### Methods returning a `status` object (or a list of objects)
#### Search method
If you seeking Twitter data to get conversations on a particular topic. This method returns a collection of relevant Tweets matching a specified query for all public tweets.

In [24]:
# most recent tweets about ETH 
tweets = api.search(q="Beyond Meat", lang="en") # TODO: How is it set to recent tweets?
for tweet in tweets:
    print(tweet.text) # printing the first tweet

RT @livekindlyco: Are you ready to try the new @BeyondMeat and @Starbucks  #vegan meat?
https://t.co/Hpf9J9AbAO
Beyond Meat $BYND shares soared another 9% on Friday, ending the week up more than 42%. That would be the biggest w… https://t.co/eslXC1Gh3z
RT @livekindlyco: Are you ready to try the new @BeyondMeat and @Starbucks  #vegan meat?
https://t.co/Hpf9J9AbAO
RT @business: The growing threat of a meat shortage is helping to lift shares of plant-based protein manufacturer Beyond Meat https://t.co/…
@WTAE Don’t worry, the shelves will be full of Al Gores meatless meat! He going to make billions from Beyond Meat. https://t.co/pktIV56tPB
Dozens of articles about meat shortages, and contaminated meat plants around the world.  Maybe McDonalds inks a Bey… https://t.co/tGvTxOCrZt
🙄chipotle need to get beyond meat
@BethBombara @gizellie I made a breakfast sandwich too! Tofu, Beyond Meat sausage, and vegan mayo on whole wheat. https://t.co/5OmBTx9Lu2
@Slab2DaBone It’s been 4 months. I’m start

The `status` object:

In [35]:
pprint(tweets[0]) # for the first tweet

Status(_api=<tweepy.api.API object at 0x000001EFF3956E08>, _json={'created_at': 'Sun Apr 26 14:19:22 +0000 2020', 'id': 1254414771706834945, 'id_str': '1254414771706834945', 'text': 'RT @livekindlyco: Are you ready to try the new @BeyondMeat and @Starbucks  #vegan meat?\nhttps://t.co/Hpf9J9AbAO', 'truncated': False, 'entities': {'hashtags': [{'text': 'vegan', 'indices': [75, 81]}], 'symbols': [], 'user_mentions': [{'screen_name': 'livekindlyco', 'name': 'LIVEKINDLY', 'id': 861093449080033281, 'id_str': '861093449080033281', 'indices': [3, 16]}, {'screen_name': 'BeyondMeat', 'name': 'Beyond Meat', 'id': 569714177, 'id_str': '569714177', 'indices': [47, 58]}, {'screen_name': 'Starbucks', 'name': 'Starbucks Coffee', 'id': 30973, 'id_str': '30973', 'indices': [63, 73]}], 'urls': [{'url': 'https://t.co/Hpf9J9AbAO', 'expanded_url': 'https://bit.ly/3cKUuuQ', 'display_url': 'bit.ly/3cKUuuQ', 'indices': [88, 111]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<

In [26]:
pprint(tweets[0]._api) # for the first tweet

In [27]:
pprint(tweets[0]._json) # for the first tweet

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Apr 26 14:19:22 +0000 2020',
 'entities': {'hashtags': [{'indices': [75, 81], 'text': 'vegan'}],
              'symbols': [],
              'urls': [{'display_url': 'bit.ly/3cKUuuQ',
                        'expanded_url': 'https://bit.ly/3cKUuuQ',
                        'indices': [88, 111],
                        'url': 'https://t.co/Hpf9J9AbAO'}],
              'user_mentions': [{'id': 861093449080033281,
                                 'id_str': '861093449080033281',
                                 'indices': [3, 16],
                                 'name': 'LIVEKINDLY',
                                 'screen_name': 'livekindlyco'},
                                {'id': 569714177,
                                 'id_str': '569714177',
                                 'indices': [47, 58],
                                 'name': 'Beyond Meat',
                                 'screen_name': 'BeyondMeat'},
    

List of tweeter accounts, useful for the following task

In [31]:
account_list = df_merged_all['twitter handle'].tolist()
print('First 3 elements:', account_list[:3])
print('Number of twitter account studied:', len(account_list))

ValueError: DataFrame constructor not properly called!

### Methods returning a `user` object (or a list of objects)
- `me` returns the authenticated user's information

In [32]:
api.me()

User(_api=<tweepy.api.API object at 0x000001EFF3956E08>, _json={'id': 1230145588659998721, 'id_str': '1230145588659998721', 'name': 'Andreas Eckmann', 'screen_name': 'AndreasEckmann', 'location': '', 'profile_location': None, 'description': '', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 0, 'friends_count': 1, 'listed_count': 0, 'created_at': 'Wed Feb 19 15:02:28 +0000 2020', 'favourites_count': 0, 'utc_offset': None, 'time_zone': None, 'geo_enabled': False, 'verified': False, 'statuses_count': 0, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': 'F5F8FA', 'profile_background_image_url': None, 'profile_background_image_url_https': None, 'profile_background_tile': False, 'profile_image_url': 'http://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png', 'profile_image_url_https': 'https://abs.twimg.com/sticky/default_profile_images/default_

- The `get_user` method returns information about the specified user.

In [33]:
target='BeyondMeat' #'BeyondMeat'
user = api.get_user(target) # argument = id, user_id, screen_name
pprint(user._json)

{'contributors_enabled': False,
 'created_at': 'Thu May 03 05:40:43 +0000 2012',
 'default_profile': False,
 'default_profile_image': False,
 'description': "Meat that's made from plants, made for meat lovers. \n"
                '#GoBeyond',
 'entities': {'description': {'urls': []},
              'url': {'urls': [{'display_url': 'beyondmeat.com',
                                'expanded_url': 'http://www.beyondmeat.com',
                                'indices': [0, 22],
                                'url': 'http://t.co/4ZbzGc2iMf'}]}},
 'favourites_count': 18113,
 'follow_request_sent': False,
 'followers_count': 108771,
 'following': True,
 'friends_count': 692,
 'geo_enabled': True,
 'has_extended_profile': False,
 'id': 569714177,
 'id_str': '569714177',
 'is_translation_enabled': False,
 'is_translator': False,
 'lang': None,
 'listed_count': 1130,
 'location': 'El Segundo, CA',
 'name': 'Beyond Meat',
 'notifications': False,
 'profile_background_color': 'FAF5E2',
 'profile

Some attributes of the `user` object:

In [38]:
print("Name:", user.name)
print("Screen name:", user.screen_name)
print("Number of followers:" ,  user.followers_count)
pprint("description: " + user.description)
pprint("Number of tweets published: " + str(user.statuses_count))
pprint("friends_count: " + str(user.friends_count))

Name: Beyond Meat
Screen name: BeyondMeat
Number of followers: 108771
" description: Meat that's made from plants, made for meat lovers. \n#GoBeyond"
' Number of tweets published: 24010'
' friends_count: 692'


**Step 1b: continuous `y` variables** [week 2]
- Fetching the data: 
    - if you work on the suggested idea, you can easily access daily stock prices using the [`yfinance` package](https://pypi.org/project/yfinance/)  (see below)
    - otherwise, you can find some interesting data listed in the syllabus

**Step 1c: merge `X_num` and `y`** [week 2]
- Beware of the temporality: in the case of the proposed study on stock market prices, you will have to deal with the fact that the X is at the tweet level while `y` is daily. 

***Milestone 1 - March 24th*** *You can submit the previous steps as a first notebook.*

**Sample split.** Do the standard 80% / 20% training/test split using all days in the data. In addition, do a separate temporal split where the training set is the first 80% of days in the time series.

**For all machine learning models**, report performance measure in test and train samples.

**Step 2a: estimate different regression models using `X_num` and `y`** [week 3]

***Milestone 2 - March 31th***  *You can submit the previous steps as a second notebook.*

**Step 3: text analysis** [week 4]
- Featurize tweets (or another text dataset related to your subject): transform the text into a standard document-level dataset `X_doc`

**Step 2b: estimate different regression models using `X_doc` and `y`** [week 3]

**Step 4: estimate a classification models** [week 5]
- propose a categorical variable `y_calc` that you can compute from the continuous one (`y`) (e.g. positive or negative growthin stock prices). For the `X` dimension, you can use `X_doc` or `X_num` or both. 
- you can use any other categorical variable that you find relevant 

***Milestone 3 - April 21th***  *You can submit the previous steps as a third notebook.*

**Step 6: Dimension reduction** [week 6]
- Use one of the dimension reductions methods to dimension-reduce the features
    - PCA or topic model (LDA or STM) or k-means clustering on the featurized text `X_doc`
- Run another classifier

**Step bonus: Econometric identification** [week 8]
- Find an exogenous shock affecting this firm (but not all the firms) and a control group of firms not affected
    - example: a natural disaster/shock to the exchange rate/change in ownership... affects the functionning of this firm but not the other firms of the stock market index.  
- scale up the previous data collection to the firms in the control group
- use one of the technique studied in class to causaly identify the impact of the exogenous shock on the stock market of the affected firm

***Milestone 4 - May 17th*** *You can submit the previous steps as a last notebook.*

---------------------------
**Requirement for completion grade**: 
- the homework should be done in **groups of 1 or 2 persons** 
- the homework will have to be submitted as a (Python or R) **notebooks**
- you can give them back at the indicated **milestone** or on May 17th (no homework will be accepted after May 17th)
- the notebooks should contain at least **three graphs** (overall)
- each notebook should run from the beginning to then end, but:
    - beware of paths to folders that are on your computer but not on mine
    - if you have such a path, you can: 
        - have the data downloaded in the notebook directly
        - import your data in a github (or any other online storage) folder that the notebook can access  
    - indicate if the notebook takes more than 30 minutes to run
- do not write us emails regarding the homework, ask your questions on the **forum**, it will benefit everyone for sure! 

---------------------------